In [118]:
!export http_proxy=
import katdal
import h5py
import numpy as np
import matplotlib as plt
import xarray as xr
import pandas as pd
import pylab as plt
import time as tme

In [5]:
# Reading in the visibility file and the flag file
visfile = katdal.open('/scratch2/isaac/1532311489_sdp_l0.rdb')
flagfile =h5py.File('/scratch2/isaac/1532311489_sdp_l0_flags.h5')

In [191]:
# Reading in the full visibity file
fullvis = katdal.open('/scratch2/isaac/1532311489_sdp_l0.full.rdb')

In [193]:
# Selecting data from the full visibility file
fullvis.select(pol = 'HH',corrprods='cross',scans='track')

In [194]:
# Getting the azmuth and elevation
az = fullvis.az
el = fullvis.el

In [195]:
azmean = (np.array([np.mean(az[:][i]) for i in range(az.shape[0])])).astype(int)
elmean = (np.array([np.mean(el[:][i]) for i in range(el.shape[0])])).astype(int)

In [10]:
def select(visfile,flagfile, pol_to_use, corrprod,scan):
    '''
    Selecting the correlection products.
    
    Input: polirization product to use, corr products type(i.e cross or auto) and type of scans
    
    Output: visibity file and flag file
    '''
    return visfile.select(corrprods = corrprod, pol = pol_to_use, scans = scan)

In [11]:
select(visfile,flagfile,pol_to_use = 'HH',corrprod='cross',scan='track')

In [24]:
def apply_flags(visfile,flagfile):
    from dask import array as da
    '''
    This function is going to apply flag table to the data.
        
    Input : Flag file and data file
    
    Output : The flag table
    '''
    flags = da.from_array(flagfile['flags'], chunks=(1, 342, visfile.shape[2]))
    visfile.source.data.flags = flags

    flagP = visfile.flags[:, :, :]
    return flagP

In [197]:
all_flags = apply_flags(visfile,flagfile)

In [93]:
def get_time_idx(visfile):
    import datetime
    '''
    This function is going to convert unix time to hour of a day
    
    Input : h5 obeject
    
    Output : list with time dumps converted to hour of a day
    '''
    unix  = visfile.timestamps

    local_time = []
    for i in range(len(unix)):
        local_time.append(datetime.datetime.fromtimestamp((unix[i])).strftime('%H:%M:%S'))

    # Converting time to hour of a day
    hour = []
    for i in range(len(local_time)):
        hour.append(int(round(int(local_time[i][:2]) + int(local_time[i][3:5])/60 + float(local_time[i][-2:])/3600)))
    return np.array(hour)[None,:]

In [94]:
time_idx = get_time_idx(visfile)

In [88]:
def get_az_idx(azmean,bins):
    '''
    This function is going get the index of the azimuth 
    
    Input : List of Azimuthal angle and azimuthal bins
    
    Output : Azimuthal index
    '''
    az_idx = []
    for az in azmean:
        for j in range(len(bins)-1):
            if bins[j] <= az < bins[j+1]:
                az_idx.append(j)
    
    return np.array(az_idx)[None,:]

In [89]:
az_idx = get_az_idx(azmean,np.arange(0,370,15))

In [91]:
def get_el_idx(elmean,bins):
    '''
    This function is going get the index of the elevation
    
    Input : List of elevation angle and bins
    
    Output : Elevation index
    
    '''
    el_idx = []
    for el in elmean:
        for j in range(len(bins)-1):
            if bins[j] <= el < bins[j+1]:
                el_idx.append(j)
    
    return np.array(el_idx)[None,:] 

In [92]:
el_idx = get_el_idx(elmean,np.arange(0,100,10))

In [100]:
def get_corrprods(visfile):
    '''
    This function is getting the corr products
    
    Input : Visibility file
    
    Output : Correlation products
    '''
    bl = visfile.corr_products
    bl_idx = []
    for i in range(len(bl)):
        bl_idx.append((bl[i][0][0:-1]+bl[i][1][0:-1]))
            
    return np.array(bl_idx)

In [101]:
corr_prods = get_corrprods(visfile)

In [189]:
def get_bl_idx(corr_prods):
    '''
    This function is getting the index of the correlation products
    
    Input  : Correlation products
    
    Output : Baseline index
    '''
    nant = 64
    A1, A2 = np.empty(nant*(nant-1)/2, dtype=np.int32), np.empty(nant*(nant-1)/2, dtype=np.int32)
    k = 0
    for i in range(nant):
        for j in range(i+1,nant):
            A1[k] = i
            A2[k] = j
            k += 1

    corr_products = np.array(['m{:03d}m{:03d}'.format(A1[i], A2[i]) for i in range(len(A1))])


    df = pd.DataFrame(data=np.arange(2016), index=corr_products).T
    
    bl_idx = df[corr_prods].values
    return (bl_idx[0])[:,None]

In [190]:
bl_idx = get_bl_idx(corr_prods)

In [192]:
#Initializing the master array and the weghting
master = np.zeros((24,4096,2016,10,24),dtype=np.uint16)
co = np.zeros((24,4096,2016,10,24),dtype=np.uint16)

In [ ]:
# Updating the array
master[time_idx,:,bl_idx,el_idx,az_idx] += np.transpose(all_flags, axes=[2,0,1])
co[time_idx,:,bl_idx,el_idx,az_idx] += 1

In [ ]:
# Computing the probabilty
pb = np.nan_to_num(master/co)

In [169]:
# Creating Xarray for ease access
ds = xr.DataArray(pb,dims = ('time','frequency','baseline','elevation','azimuth'))